In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/train.csv')
val = pd.read_csv('/content/gdrive/MyDrive/val.csv')

In [ ]:
train = train[train['Class']!='Human']
val = val[val['Class']!='Human']

In [ ]:
X_train = train[['Text', 'len in symbols',
       'len in tokens', 'vowels / symbols', 
       'mean word length', 'long words / all words',
       'small words', 'Dale-Chall', 'Gunning-Fog', 'Flesch',
       'func_pos/all_words', 'cont_pos/all_words', 'noun_pos/all_words',
       'verb_pos/all_words', 'noun_pos/verb_pos']]
y_train = train['Class']

In [ ]:
X_test = val[['Text', 'len in symbols',
       'len in tokens', 'vowels / symbols', 'mean word length', 'long words / all words',
       'small words', 'Dale-Chall', 'Gunning-Fog', 'Flesch',
       'func_pos/all_words', 'cont_pos/all_words', 'noun_pos/all_words',
       'verb_pos/all_words', 'noun_pos/verb_pos']]
y_test = val['Class']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

randomforest = RandomForestClassifier()
logreg = LogisticRegression()

In [ ]:
randomforest.fit(X_train.drop('Text', axis=1), y_train)
y_pred = randomforest.predict(X_test.drop('Text', axis=1))
accuracy_score(y_test, y_pred)

0.2725032725032725

In [ ]:
df = pd.DataFrame({'texts':X_test['Text'], 'predictions': le.inverse_transform(y_pred), 
                   'labels':le.inverse_transform(y_test)})

In [ ]:
df

,texts,predictions,labels
0,Как насчёт ещё одного раунда?,M-BART50,OPUS-MT
2,Соловьев рассказал о просмотре в эфире одного ...,ruGPT2-Large,ruGPT3-Large
3,"Доброе утро, девочки и мальчики",M2M-100,mT5-Small
4,В октябре 2005 — докладчица на Европейском Кон...,M-BART50,M2M-100
5,"Также большая проблема состоит в том, что мы н...",ruGPT3-Medium,ruGPT3-Small
...,...,...,...
21503,Чуть не культовая лента — «Плетеный человек» к...,ruGPT3-Medium,ruGPT3-Large
21506,Строев обвиняет в слабости центральных властей...,OPUS-MT,M2M-100
21507,"Стерильны бывают и мужчины, но религиозная док...",mT5-Large,M-BART
21508,"Целый день в Pokémon Stadium, а потом еще и на...",ruGPT3-Large,ruGPT3-Large


In [ ]:
df.to_csv('Random_Forest_features.csv')

In [ ]:
logreg.fit(X_train.drop('Text', axis=1), y_train)
y_pred = logreg.predict(X_test.drop('Text', axis=1))
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.21791021791021792

In [ ]:
df = pd.DataFrame({'texts':X_test['Text'], 'predictions': le.inverse_transform(y_pred), 
                   'labels':le.inverse_transform(y_test)})
df.to_csv('LogReg_features.csv')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch import optim
from torch.nn.utils.rnn import pad_sequence

In [ ]:
class MLP(nn.Module):
    
    def __init__(self, num_classes, embedding_dim=14):
        
        super().__init__()          
        # указываем в атрибутах класса, какие слои и активации нам понадобятся
        self.emb2h = nn.Linear(embedding_dim, 200) 
        self.act1 = nn.ReLU()
        self.h2h = nn.Linear(200, 100) 
        self.act1 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.h2out = nn.Linear(100, num_classes)
        
        
    def forward(self, text): #необходимый метод,  в нем указываем, как именно связываются слои/активации между собой
        # batch_size x seq_len
        #ipdb.set_trace()
        hidden = self.emb2h(text) # пропускаем эмбеддинг через полносвязный слой 
        # batch_size x 10
        hidden = self.act1(hidden)
        hidden = self.h2h(hidden) # пропускаем эмбеддинг через полносвязный слой 
        # batch_size x 10
        hidden = self.act1(hidden)
        # batch_size x 10
        hidden = self.dropout(hidden)
        # batch_size x 10
        out = self.h2out(hidden) # возвращаем одно число для каждого семпла
        # batch_size x 1# пропускаем число через сигмоиду, делая из него вероятность класса
        # batch_size x 1
        return out

In [ ]:
class Dataset(Dataset):
    def __init__(self, matrix, labels, DEVICE='cuda:0'):
        self.matrix = matrix.drop(['Text'], axis=1)
        self.texts = matrix['Text']
        self.length = matrix.shape[0]
        self.target = labels
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        ids = torch.Tensor(self.matrix.iloc[index])
        y = self.target[index]
        texts = self.texts.iloc[index]
        return ids, y, texts

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
train_data = Dataset(X_train, y_train, 'cpu')
val_data = Dataset(X_test, y_test, 'cpu')
train_iterator = DataLoader(train_data, shuffle=True, batch_size=32)
val_iterator = DataLoader(val_data, shuffle=True, batch_size=32)

In [ ]:
def train(model, iterator, optimizer, criterion):
    print('Training...')
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах
    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (features, ys, texts) in enumerate(iterator): #итерируемся по батчам
        features = features.to('cuda:0')
        ys = ys.to('cuda:0')
        optimizer.zero_grad()  #обнуляем градиенты
        #ipdb.set_trace()
        preds_proba = model(features) #прогоняем данные через модель
        loss = criterion(preds_proba, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        
    print(f'Train loss: {epoch_loss/i}')
    return epoch_loss / len(iterator) # возвращаем среднее значение функции потерь по всей выборке

In [ ]:
def evaluate(model, iterator, criterion):
    print("\nValidating...")
    epoch_loss = 0
    accuracy = []
    model.eval()
    pred = []
    labels = [] 
    text = []
    with torch.no_grad():
        for i, (features, ys, texts) in enumerate(iterator):   
            features = features.to('cuda:0')
            ys = ys.to('cuda:0')
            predictions = model(features)  # делаем предсказания на тесте
            loss = criterion(predictions, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item() 
            accuracy.append(accuracy_score(ys.cpu().detach().numpy(), predictions.argmax(axis=1).cpu().detach().numpy()))
            pred.extend(predictions.argmax(axis=1).cpu().detach().tolist())
            labels.extend(ys.cpu().detach().tolist())
            text.extend(list(texts))
    print(sum(accuracy)/len(accuracy))
    return epoch_loss / len(iterator), pred, labels, text # возвращаем средний лосс по батчам

In [ ]:
model = MLP(num_classes=13)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to('cuda:0')
criterion = criterion.to('cuda:0')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
from tqdm.notebook import tqdm
losses = []
losses_eval = []

for i in tqdm(range(20)):
    print(f'\nstarting Epoch {i}')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)

    epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)

  0%|          | 0/20 [00:00<?, ?it/s]


starting Epoch 0
Training...
Train loss: 1.9540821232556709

Validating...
0.24290731618317823

starting Epoch 1
Training...
Train loss: 1.9237741685442584

Validating...
0.2509806604634191

starting Epoch 2
Training...
Train loss: 1.9070039788228914

Validating...
0.2554306923918993

starting Epoch 3
Training...
Train loss: 1.8980741220707764

Validating...
0.25394829866812624

starting Epoch 4
Training...
Train loss: 1.8927259107251195

Validating...
0.25799922459405217

starting Epoch 5
Training...
Train loss: 1.8888661020979494

Validating...
0.2610153256704981

starting Epoch 6
Training...
Train loss: 1.8843393127939794

Validating...
0.2648011311804415

starting Epoch 7
Training...
Train loss: 1.8818253537819047

Validating...
0.2657247765006385

starting Epoch 8
Training...
Train loss: 1.8792855508423478

Validating...
0.2687123700054735

starting Epoch 9
Training...
Train loss: 1.8781660065458952

Validating...
0.25872046615581096

starting Epoch 10
Training...
Train loss: 1.8

In [ ]:
epoch_loss_on_test, predictions, labels, texts = evaluate(model, val_iterator, criterion)


Validating...
0.26106378854223683


In [ ]:
df = pd.DataFrame({'texts':texts, 'predictions':le.inverse_transform(predictions), 'labels':le.inverse_transform(labels)})

In [ ]:
df.to_csv('FNN_features.csv')